### 5.
O processamento de informação geográfica é uma ferramenta omnipresente no nosso dia a dia. Pretende-se
estudar treinos associados a vários desportos. Foi criado um dataset associados a vários utilizadores da aplicação
EndoMondo. Serão disponibilizado vários ficheiros json onde será possível explorar a dinâmica dos vários treinos.
Pretende-se que sejam explorados usando a biblioteca ipyleaflet. Pretende-se que seja desenhadas as etapas,
calculados o nº de km a subir e a descer, determinada a velocidade média (em todo o percurso e separadamente
nos troços a subir e a descer), a distância em plano, etc. Devem comparar a evolução do ritmo cardíaco em
vários percursos e relaciona-lo com as zonas de treino 